## Idea of automizing the search for RNA-seq data within BioStudies. 
### many cases:
### - ENA got the data (fastq (i need to calc it myself somewhen ...))
### - count matrix is attached to files 
### - probably more of this ... 

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import sys
from scipy.io import mmread
from utils.helper import get_negative_values, METADATA_COLS, reorder_columns_by_metadata_and_gene_counts, convert_gene_names_to_ensembl, clean_age
from utils.plotting import violinplot_overall, scatter_plot, manhattanplot
import requests


In [ ]:
url_get_study="https://www.ebi.ac.uk/biostudies/api/v1/studies/"

ena_url_filereport_ERP = "https://www.ebi.ac.uk/ena/portal/api/filereport?result=read_run&accession="
ena_url_query_end = "&format=json&fields=study_accession,secondary_study_accession,sample_accession,secondary_sample_accession,experiment_accession,run_accession,submission_accession,tax_id,scientific_name,instrument_platform,instrument_model,library_name,nominal_length,library_layout,library_strategy,library_source,library_selection,read_count,base_count,center_name,first_public,last_updated,experiment_title,study_title,study_alias,experiment_alias,run_alias,fastq_bytes,fastq_md5,fastq_ftp,fastq_aspera,fastq_galaxy,submitted_bytes,submitted_md5,submitted_ftp,submitted_aspera,submitted_galaxy,submitted_format,sra_bytes,sra_md5,sra_ftp,sra_aspera,sra_galaxy,sample_alias,broker_name,sample_title,nominal_sdev,first_created,bam_ftp,bam_bytes,bam_md5"
ena_tsv_download_query_end= '&fields=study_accession,sample_accession,experiment_accession,run_accession,tax_id,instrument_platform,instrument_model,library_strategy,base_count,center_name,experiment_title,fastq_ftp,submitted_ftp,sra_ftp,sample_title&format=tsv&download=true&limit=0'
ena_json_query_end= '&fields=study_accession,sample_accession,experiment_accession,run_accession,tax_id,instrument_platform,instrument_model,library_strategy,base_count,center_name,experiment_title,fastq_ftp,submitted_ftp,sra_ftp,sample_title&format=json&limit=0'


###### ENA handling
#TODO check if this is even needed. right now the endpoint does not work anyways
# maybe we can also just directly download the TSV and the downloading script (Download all button for fastq files... this then downloads a lot of data if we shall execute it)
# Source https://www.ebi.ac.uk/ena/browser/api/swagger-ui/index.html#/content-controller/getSummary 

def get_ena_link(study_json):
    ena_links = []
    for link in study_json.get('section', {}).get('links', []):
        if link.get('url').startswith('ERP'):
            ena_links.append(link.get('url'))
    return ena_links

def download_ena_tsv(ena_accession):
    response = requests.get(ena_url_filereport_ERP + ena_accession + ena_tsv_download_query_end)
    if response.status_code == 200:
        file_path = os.path.join(ena_accession, f'{ena_accession}.tsv')
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"File downloaded and saved as '{file_path}'.")

    else:
        print(f"Failed to retrieve data: {response.status_code}")

### this is same as the tsv but in json format
def get_ena_fastq_ftp_download_links(ena_accession):
    fastq_links = []

    response = requests.get(ena_url_filereport_ERP + ena_accession + ena_json_query_end)
    if response.status_code == 200:
        data = response.json()
        for sample in data:
            sample_ftp_links = sample['fastq_ftp'].split(';')
            fastq_links = fastq_links + sample_ftp_links

    else:
        print(f"Failed to retrieve data: {response.status_code}")

    file_path = os.path.join(ena_accession, f'{ena_accession}_fastq_ftp_links.txt')
    with open(file_path, 'w') as f:
        for link in fastq_links:
            f.write(link + '\n')


#TODO delete?
def get_filereport(ena_accession):
    ena_accession_primary = []

    requests.get(ena_url_filereport_ERP + ena_accession + ena_url_filereport_ERP)
    if response.status_code == 200:
        data = response.json()
        # now as we have gotten to the overview: 
        # TODO here we could automate the download for every sample ID in the filereport (without limit should show it all)
        
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        
        
def ena_handling(ena_accession):
    os.makedirs('ENA' + ena_accession, exist_ok=True)
    print(f"Directory '{ena_accession}' created or already exists.")

    download_ena_tsv(ena_accession)
    get_ena_fastq_ftp_download_links(ena_accession)

In [ ]:
def get_biosample_query_response(biosample_url):
    response = requests.get(biosample_url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return 0
        
        

def is_useful_file(filename):
    filename = filename.lower()
    
    # Check if "count" is in the filename or if it ends with .fastq or .fq
    if "count" in filename:
        return True
    
    if filename.endswith(".fastq") or filename.endswith(".fq"):
        return True
    
    return False

def filter_study_characteristics(study_json):
    useful_files = []
    is_useful = False
    # Navigate to the 'Source Characteristics' subsection
    for subsection in study_json.get('section', {}).get('subsections', []):
        #TODO bugfix, in example the first is an array ... 
        if type(subsection) is not dict:
            continue
        if subsection.get('type') == 'Source Characteristics':
            organism = None
            organism_part = None
            for attribute in subsection.get('attributes', []):
                if attribute.get('name') == 'Organism':
                    organism = attribute.get('value')
                elif attribute.get('name') == 'Organism part':
                    organism_part = attribute.get('value')
            # Check if conditions are met
            if organism == "Homo sapiens" and organism_part in ["blood", "whole blood", "PBMC"]:
                is_useful = True
            
        #TODO this is prone, sometimes processed data appears under another hierarchy 
        if subsection.get('type') == 'Processed Data':
            for files in subsection.get('files', []):
                if is_useful_file(files.get('path')):
                    useful_files.append(files.get('path'))
            
    return is_useful, useful_files

In [ ]:
#NOTE here i want to store some possible search queries, I produced the queries there and copied with inspect tool of the browser:  https://www.ebi.ac.uk/biostudies/ 

#LIMIT
query_minimal = '%28RNA-seq+or+rnaseq%29+AND+%28human+or+%27homo+sapiens%27%29'   # ~3k results
query_medium = '%28RNA-seq+or+rnaseq%29+AND+%28human+or+%27homo+sapiens%27%29+AND+%28blood+OR+%27whole+blood%27+OR+PBMC%29+AND+NOT+%27single+cell%27+AND+NOT+%27cord+blood%27' # 630 results
query_long="%28rna-seq+OR+rnaseq%29+AND+%28human+OR+%22homo+sapiens%22%29+AND+%28blood+OR+%22whole+blood%22+OR+PBMC%29+AND+%28%22bulk%22%29+AND+NOT+%28sc+OR+%22single-cell%22%29+AND+NOT+DNA"
## pageSize 100 is max
current_page = 1
url_get_all = f"https://www.ebi.ac.uk/biostudies/api/v1/search?pageSize=100&page={str(current_page)}&query="

query = query_medium

##
url = url_get_all + query
print(url)
data = get_biosample_query_response(url)
hits = data.get("hits", [])
total_hits = data.get("totalHits")

print(f'{len(hits)} / {total_hits}')

filtered_hits = [hit for hit in hits if "GEO" not in hit["accession"]]
print(f'{len(filtered_hits)}')
# Display the filtered results
for hit in filtered_hits:
    
    response = requests.get(url_get_study + hit['accession'])
    if response.status_code == 200:
        data = response.json()
        
        is_useful, useful_files = filter_study_characteristics(data)
        print(f"Useful: {is_useful}")           
        if is_useful:
            print("="*50)            
            print(f"Accession: {hit['accession']}")
            print(f"Title: {hit['title']}")
            print(f"Views: {hit['views']}")
            print(f"Useful files: {useful_files} for {hit}")
            print("="*50)            

            ena_accession = get_ena_link(data)
            print(f'{ena_accession}')
        
            #NOTE ENA handling 
            ena_handling(ena_accession)
    else:
        print(f"Failed to retrieve data: {response.status_code}")
    
    if total_hits - len(hits) > 0:
        current_page += 1
        #TODO next 100 pages ... 
        
        
    time.sleep(0.2)
